# *MoMA's* artists and their collaboration network
## Data project for "2021W 136010-1 Introduction to DH Tools and Methods". 
##### Nora Linser (01315533) 2022-01-23

# Introduction
*The Museum of Modern Art* in New York provides two datasets on *github*, one containing the artists that are represented in their collections and the other one the artworks itselt. 
Ithe artworks themselves. 
I will use the datasets, particularly the artwork dataset, to conduct a social network analysis of the artists represented in the *MoMA* collections. In doing so, I will assume that there is a social connection between the artists who collaborated in the creation of an artwork. 

# Research question and objective
The goal of this project is to examine the dataset and create a network analysis based on the shared authorship of artworks by artists. 
A graph object will be created and centrality measures (Betwenness Centrality, Closeness Centrality, Degree distribution) will be calculated and displayed in plots (histograms and graphs). The size of the nodes in the respective diagrams should depend on the displayed centrality measures. Based on one of the centrality measures, a subgraph containing only the 100 most important nodes should be computed. A nice representation of this subgraph is the goal of the network investigation, where also some of the node attributes like gender or origin should be reflected (<font color =darkred>this part is not finished yet, but will be fulfilled in the Final Assignment</font>). 

# Source
*MoMA* notes in the README file on Github that some of the data is not complete and that other information is not "approved by the curator." They also make it clear that use of the data is at the user's risk. 
MoMA planned to update the records on Github monthly, but the last update was done in January 2021. This is also the version (1.62) that I downloaded to use for this project.

## Reference:  
- Data collected by: Moma – Museum of modern Art.
- Dataset: 15,222 records. Encoding: UTF-8. 
- Data format: .csv and JSON.
- Data distribution via [Github](https://github.com/MuseumofModernArt/collection) by the users John Halderman and momadm. 
- Licensing: CC0 License.
- Digital object identifier DOI: 10.5281/zenodo.4408594
- [Presidential URL](https://zenodo.org/record/4408594#.YcGKvC1h1pR)
- Version: v1.62, release date: 2021/01/01
- Day and time of download: 2021/12/21 12:43 PM 

## Short citation:
Moma – Museum of Modern Art (2021/01/01), Artists (Data file in CSV Format). Doi: 10.5281/zenodo.4408594. Retrieved from https://github.com/MuseumofModernArt/collection  

Moma – Museum of Modern Art (2021/01/01), Artwork (Data file in CSV Format). Doi: 10.5281/zenodo.4408594. Retrieved from https://github.com/MuseumofModernArt/collection 


# Importing the dataset and getting a first overview
Setting up the working environment:

In [ ]:
#Setting up the working environment
import pandas as pd

Importing the files and displaying the first overview:

In [ ]:
#Import the file downloaded from github
artworksAll = pd.read_csv("/Users/linsernora/Pyhton_Course/Python_CourseNorLins/MoMAartworks/Artworks.csv")
artworksAll.head(15)

My first attemt to find out if there are objects listed with more than one artist in the Artist column was no very helpful.
I though my just printing a couple of rows I might get an impression of the data. 

Getting all the column labels.

In [ ]:
#listing all the column headers of the dataset: 
list(artworksAll.columns.values)

## Finding multiple artist occurrences

I tried to find out if there are rows with more than on artist by using the .isin() method. 
At first I though that the given result would suggest that there are no commas in the column. 
To use this approach I would have needed to take further steps with this result and to find out in which columns the result is "True". 

In [ ]:
#checking if there are rows with multiple artists in the Artist column
artists = artworksAll["Artist"]
multiple = artists.isin([","])
multiple.head()


The next apprach was more useful. Subsetting the rows that contain a "," in the "Artist" column and assigning it to a new DataFrame *multipleArtists*.
The .info() methods shows that there are 8059 entries in the dataset where there is more than one artist mentioned in the "Artist" column. 

In [ ]:
#Subsetting the rows with more than one artist. 
multipleArtists = artworksAll[artworksAll["Artist"].str.contains(",")==True]
multipleArtists.info()

Before geoing any further I wanted to check if the dataset is restricted to show one object per row. As the result shows, there are no duplicate "ObjectID"'s in the dataset. 

In [ ]:
#checking if there are objectsIds mentioned more than once: 
objectIds = artworksAll["ObjectID"]
artworksAll[objectIds.isin(objectIds[objectIds.duplicated()])]


## Cleaning the data
Reducing the columns that are not needed for the task.

In [ ]:
#reducing the dataframe, only keeping the columns that are needed: 
multipleArtists = multipleArtists[["Title", "Artist", "ConstituentID", "ObjectID"]]

In [ ]:
#checking if it worked: 
multipleArtists.head()

In [ ]:
multipleArtists.info()

### Splitting up the values
Next step is to split up the values from the columns "Artist", "Constitutent ID" and "Title" into separate rows. 
So that each artist occurrence is mentioned in its own row. The data in the columns "Title" and "ObjektID" needs to be copied to the new observations. 

With a little help of *stackoverflow* I created the following approach of stacking and unstacking that provides me with the desired output. I created a new dataframe with the result called *singleArtist*.

In [ ]:
#Stacking and unstacking the data to move every artist-occurrence from the "Artist" column into separate lines. 
#Without loosing the data that needs to be copied. 
singleArtist = (multipleArtists.set_index(['Title', 'ObjectID'])
   .stack()
   .str.split(',', expand=True)
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index()
)

In [ ]:
#calling singleArtist to see if it worked: 
singleArtist

Investing if the first rows do not hold any data in the Title column or if something went wrong there. 

In [ ]:
#chechking if the object does not have a title of if I made I mistake: 
multipleArtists[multipleArtists["ObjectID"] == 136435]

The artwork does not hold any data as Titel.
Making sure the stacking worked and also the constiutent IDs were split up accordingly to the artist names. 

In [ ]:
#checking if I split up the Constitutent IDs correctly
singleArtist[singleArtist["ObjectID"] == 81]

In [ ]:
singleArtist[singleArtist["Artist"] == "Peter Eisenman"]

The multiple occurrences in the "Artist"Column are split up into separate lines, the metadata was copied in to the lines as wanted. 

Before adding the additional rows for each artist, the dataframe contained 'r: mutipleArtists.count()' 8059 elements. Now, after splitting up the multiple occurrences, the dataframe holds 24900 elements. 
This might be to big to comfortable work with in my environment, but I will try it out. 

## Creating a Graph object
To investigate the network and to be able to draw graphs

Creating a nodelist with unique values

In [ ]:
#creating a unique list of all the possible nodes, I might need that later.
objectEdges = singleArtist["ObjectID"].unique()
objectEdges

In [ ]:
#how many objects are we dealing with in the dataset singleArtist?
print("Count of objectnodes in the dataframe singleArtist:", len(objectnodes))

In [ ]:
#creating a unique list of nodes for the ArtistID's
artistnodes = singleArtist["ConstituentID"].unique()
#how many unique artists are in the singleArtist dataframe?
print("Count of artistnodes in the dataframe singleArtist:", len(artistnodes))

Importing networkx and matplotlib packages for further investigation and vizualisation of the social network. 

In [ ]:
import networkx as nx
from networkx import Graph as NXGraph
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
#creating an empty graph
G = nx.Graph()

In [ ]:
#creating an edgelist containging only the ObjectIDs and the ConstitutentIDs. might be useful later.
edgelist = multipleArtists[["ObjectID", "ConstituentID"]]

In [ ]:
#adding the edges: 
G= nx.from_pandas_edgelist(edgelist, source="ObjectID", target="ConstituentID")


In [ ]:
#don't run again if not needed, takes a long time. 
#nx.draw(G)

Running this takes up to 10 minutes. To comfortably work with the dataset, I need to reduce the size.  

# Reducing the size fo the graph object
The graph object holding the complete data needs to be reduced, it slows down my computer to much. 
For this project I will limit the dataset based on the aquisition date of the object. 

Only the objects that where arquired between the years 1980 and 2000 stay in the dataset. To do this I subset the original dataframe with a slice on the "DateAquired" column. 
To do so I set the "DateAcquired" column as index and sort the dataframe based on that values. 

In [ ]:
#Setting the DateAcquired Colum as index and sorting it in ascending order. 
artworks_ind = artworksAll.set_index("DateAcquired").sort_index()
artworks_ind.head()

Next I slice the df brased on the timeframe between 1980 and 2000.

In [ ]:
#Slicing the DataFrame so we only keep objects with an aquisition date ("DataAquired") between 1980 and 2000: 
artworks_sliced1 = artworks_ind.loc["1980-01-01":"2000-01-01"]
#getting the info on the new dataset
artworks_sliced1.info()

In [ ]:
print("The new dataframe contains", len(artworks_sliced1), "elements.")

This slice holds 21682 elements and is still to big. I shorten the timeframe again to only work with objects that were aquired between the years 1980 and 1990.

In [ ]:
#reducing the slice to the timeframe 1980 until 1990
artworks_sliced = artworks_ind.loc["1980-01-01":"1990-01-01"]
artworks_sliced.info()

In [ ]:
print("The sliced dataframe contains", len(artworks_sliced), "elements")

With this dataset of 10187 lines I repeat the steps from above.  

# Prepping the smaller dataset
Repeating the steps from above: Subsetting the rows with multiple artists, removing unnecessary columns, restacking the data so every artist is mentioned in its own row, adding columns with ID an prefix "artist" or "object" repectiviley. 

In [ ]:
#Subsetting the rows with more than one artist. 
multiArtists = artworks_sliced[artworks_sliced["Artist"].str.contains(",")==True]
multiArtists.info()

In [ ]:
#only keeping the columns that are needed: 
multiArtists = multiArtists[["Title", "Artist", "ConstituentID", "ObjectID"]]

In [ ]:
#removing the multiple values and adding them as new rows:
single_Artist = (multiArtists.set_index(['Title', 'ObjectID'])
   .stack()
   .str.split(',', expand=True)
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index()
)
single_Artist.info()

In [ ]:
single_Artist.head()

In [ ]:
print("Number of elements in the dataframe single_Artist:", len(single_Artist))

This leaves us with 1559 entries, a managable size for this social network project. 

## Adding prefixes to the Ids to distinguish them: 
Before I go any further I am adding new columns to the dataset where I store acombination of the ObjectID and the ConstituentId together with the prefixes "object" and "artist". The numbers alone would not be distinguishable.

In [ ]:
#adding a new column where the ObjectId has the prefix "object"
single_Artist["ObjectID_unique"] = "object" + single_Artist["ObjectID"].astype(str)
single_Artist.head()

In [ ]:
#adding another new column where the ConstitutentID has the prefix "artist"
single_Artist["ConstituentID_unique"] = "artist" + single_Artist["ConstituentID"].astype(str)
single_Artist.head()

In [ ]:
#removing the white spaces:
single_Artist["ConstituentID_unique"] = single_Artist["ConstituentID_unique"].str.replace(' ', '')
single_Artist.head()

In [ ]:
#To make sure there are no whitespaces in the "ObjectID_unique" column
single_Artist["ObjectID_unique"] = single_Artist["ObjectID_unique"].str.replace(' ', '')
single_Artist.head()

With the cleaned up data I count the unique ObjetIds and unique ConstitutenIDs again. 


In [ ]:
#counting the unique artist Ids in the new DataFrame: 
artist_Ids = single_Artist["ConstituentID_unique"].unique()
print("There are", len(artist_Ids), "unique artists mentioned in the dataset.")

In [ ]:
#counting the unique object Ids in the new DataFrame: 
object_Ids = single_Artist["ObjectID_unique"].unique()
print("There are", len(object_Ids), "unique objects mentioned in the dataset.")

The new smaller Dataframes contains 575 artists and 615 different objects. Lets find out how they are connected to each other. 

# Adding node attributes
To do so we need to get the data from the artists dataset. We will then compute different colours for the gender values in the dataset. This column will than be appended to the graph object where it will be used to plot the node color accordingly. 

Getting the gender data from the artists dataset, to have an unique identifyer we need to adapt the ConstitutentID like we did for the graph object and add "artists" to the values as prefix. We allso remove unwanted whitespaces.

In [ ]:
#Add the "ConstitutentID_unique" column to the artis df, so that we have a unique identifier.
artists_complete["ConstituentID_unique"] = "artist" + artists_complete["ConstituentID"].astype(str)
#remove the whitespaces
artists_complete["ConstituentID_unique"] = artists_complete["ConstituentID_unique"].str.replace('_', '')
artists_complete.head()


In [ ]:
#getting the count of the unique values
len(artists_complete["Gender"].unique())

In [ ]:
#getting the unique values
artists_complete["Gender"].unique()

We need to unify the gender entries to four unique values: "non-binary", "male", "female", "nan".


In [ ]:
#remove the whitespaces:
artists_complete["Gender"] = artists_complete["Gender"].str.replace('Male', 'male')
artists_complete["Gender"] = artists_complete["Gender"].str.replace('Female', 'female')
artists_complete["Gender"] = artists_complete["Gender"].str.replace('Non-Binary', 'non-binary')
artists_complete["Gender"] = artists_complete["Gender"].str.replace('Non-binary', 'non-binary')
#checking how many unique values we have now:
artists_complete["Gender"].unique()

In [ ]:
#checking if we now have 4 unique values.
len(artists_complete["Gender"].unique())

In [ ]:
#Replace NaN values with the value "notspecified"
artists_complete["Gender"] = artists_complete["Gender"].fillna("notspecified")
artists_complete["Gender"].unique()

We now use the numpy package to add a new column "GenderColour" to the dataframe artists_complete. The colum stores colours according to one of the 4 gender values. 

In [ ]:
#https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/ 
#create list of conditions
import numpy as np
conditions = [
    (artists_complete["Gender"] == "male"), 
    (artists_complete["Gender"] == "female"), 
    (artists_complete["Gender"] == "non-binary"),
    (artists_complete["Gender"] == "notspecified")
    #(artists_complete["Gender"] == "NaN")
    ]
#create a list of the values we want to assign for each condition (order is important)
values = ["green", "blue", "red", "gray"]
#create a new column and use np.select to assign values to it using our lists as arguments
artists_complete["GenderColour"] = np.select(conditions,values)
#inspect the df
artists_complete.head()

In [ ]:
#gender_nodes = nx.get_node_attributes(Gnew, "GenderColour")

We also need to add a colour to the objects. So that all nodes will have a colour attribute.  Here all get the same colour assign to them, whichs makes it easier. 

In [ ]:
#Importing the file. 
#Import the file from github
artists_complete = pd.read_csv("/Users/linsernora/Pyhton_Course/Python_CourseNorLins/MoMAartworks/Artists.csv")
artists_complete.head(15)

In [ ]:
#adding a column to the dataframe with the colour the nodes should be displayed in the graph
single_Artist["object_colour"] = "lightgray"
single_Artist.head()

In [ ]:
#add the column to the df that was used to create the dataframe. 
#get all the nodes with prefix "artists",  add the node attribute "GenderColour" and set it to "ligthgray"

#getting the nodes that don't hold the attribute "GenderColour"

#list(Gnew.nodes.data("GenderColour"))


#we start with one attripute (gender): 
#object_colour = single_Artist[["ObjectID_unique", "object_colour"]]
#setting the index
#object_colour_dic = object_colour.set_index("ObjectID_unique").T.to_dict("list")
#adding the "gender"data to the node attributes
#nx.set_node_attributes(Gnew, object_colour_dic, "GenderColour")

#Gnew.nodes(data=True)

## Merging the datasets. 

I will now add metadata on the artist entities from the second dataset MoMA provides.

In [ ]:
#Importing the file. 
#Import the file from github
artists_complete = pd.read_csv("/Users/linsernora/Pyhton_Course/Python_CourseNorLins/MoMAartworks/Artists.csv")
artists_complete.head(15)

In [ ]:
single_Artist.head()

In [ ]:
#just to remeber the size of the single_Artist Dateframe: 
single_Artist.shape

In [ ]:
#adding the metadata based on the ConstituentID (as unique value in both dfs) 
#at first try I received the error "You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat". 
#So I convert the ConstituentID vlaues to type text (since there is also "Nan" in there).

single_Artist["ConstituentID"]=single_Artist["ConstituentID"].astype(str)
artists_complete["ConstituentID"]=artists_complete["ConstituentID"].astype(str)

In [ ]:
#merging the two dataframes based on the ConstituentID column

single_Artistextended = pd.merge(single_Artist, artists_complete, on="ConstituentID", how = "left")
single_Artistextended.head()

In [ ]:
#checking the shape of the df and making sure only columns where added: 
single_Artistextended.shape

# Creating graph object 
With the smaller dataframe a new graph object is created (Gnew).  

The graph is set up as undirected. Since there is no direction in the relation between artists and objects. 

In [ ]:
#initiate an empty undirected graph
Gnew = nx.Graph()
type(Gnew)

In [ ]:
#adding the edges to the graph. I am setting the unique object Id columns as the source and the unique artist Ids as the target.
Gnew= nx.from_pandas_edgelist(single_Artistextended, source="ObjectID_unique", target="ConstituentID_unique")


In [ ]:
#set_node_attributes() takes a dictionary as input. To assign multiple attriputes to the nodes at once, we have to create a dictionary of dictionaries. 
#the outer dictionary represents the nodes, the inner the keys korresponding to the attributes. 
#good thing: nodes that are not in the graph are ignored. 

#we start with one attripute (gender): 
artists_gender= artists_complete[["ConstituentID_unique", "GenderColour"]]
#setting the index
artists_gender_dic = artists_gender.set_index("ConstituentID_unique").T.to_dict("list")
#adding the "gender"data to the node attributes
nx.set_node_attributes(Gnew, artists_gender_dic, "GenderColour")


In [ ]:
# Drawing the first graph

In [ ]:
#plotting the first graph without any specifications: 
nx.draw(Gnew)
plt.show()

In [ ]:

#plot graph T with the node colour based on the gender. 
#plotting T with node_size depending on betweenness centrality
npos = nx.spring_layout(Gnew, scale=1)
fig = plt.figure(1, figsize=(20,20))
gendercolours = [nx.get_node_attributes(Gnew, "GenderColour")[g] for g in Gnew.nodes()]
nx.draw(Gnew, pos=npos, 
        node_size=[v * 10000 for v in between_centrality_T.values()], 
        #also adding labels
        with_labels=True,
        node_color=gendercolours,
        #node_color = [nx.get_node_attributes(Gnew,'GenderColour')[g] for g in Gnew.nodes()], 
        edge_color="black")
plt.show()

# Refining the plots
With setting some arguments the graph plot can be refined and made better readable. 

In [ ]:
#plotting the same graph but with some arguments 
pos = nx.spring_layout(Gnew, scale=1)
fig = plt.figure(1, figsize=(20,20))
nx.draw(Gnew, with_labels=False, node_color="seagreen", node_size = 15)

##### Exploring the network

In [ ]:
#Getting feeling for the size of the graph 
#Count of the nodes:
print("There are", len(Gnew.nodes()), "nodes in the network.")

In [ ]:
#Count of the edges: 
print("There are", len(Gnew.edges()), "edges in the network.")

# Investigating the centrality measure of the network: 

## Degree centrality
Finding the network centrality meassures.
Since the network is not directed, we don't need to take in degree and out degree into account. 

In [ ]:
#computing the degree centrality
centrality = nx.degree_centrality(Gnew)
#adding the degree_centrality values as node attributes
nx.set_node_attributes(Gnew, centrality, "DegreeCentrality")

In [ ]:
#plotting the degree distribution of the network
plt.title("Histogram of degree centrality distribution")
plt.hist(list(centrality.values()))
plt.show()

In [ ]:
#importing matplotlib
import matplotlib.cm as cm

In [ ]:
#drawing the graph with nodessizes based on the centrality meassure
pos = nx.spring_layout(Gnew, scale=1)
fig = plt.figure(1, figsize=(30,30))
nx.draw(Gnew, pos=pos, node_size=[v * 1000 for v in centrality.values()])
plt.show()

#### Defining the function *find_nodes_with_highest_deg_cent* 
The function eturns the nodes with the highest degree centrality in the graph. 

In [ ]:
# Defining find_nodes_with_highest_deg_cent()
def find_nodes_with_highest_deg_cent(G):
    '''returns the nodes with the higehst degree centrality in the graph G'''
    # Computing the degree centrality of G: deg_cent
    deg_cent = nx.degree_centrality(G)
    # Computing the maximum degree centrality: max_dc
    max_dc = max(list(deg_cent.values()))
    nodes = set()
    # Iterating over the degree centrality dictionary
    for k, v in deg_cent.items():
        # Checking if the current value has the maximum degree centrality
        if v == max_dc:
            # Adding the current node to the set of nodes
            nodes.add(k)
    return nodes

# Find the node(s) that has the highest degree centrality in G: top_dc
top_dc = find_nodes_with_highest_deg_cent(Gnew)
print("Those are the nodes with the highest degree centrality: ", top_dc)

In [ ]:
# Assertion statement that checks that the node(s) is/are correctly identified.
for node in top_dc:
    assert nx.degree_centrality(Gnew)[node] == max(nx.degree_centrality(Gnew).values())

## Closeness Centrality: 
Assuming that important nodes are closer to other nodes. 
*Closeness centrality* is calucalted as the sum of the path length from the given node to all other nodes. 

In [ ]:
#calculating the closeness centrality:
close_centrality = nx.closeness_centrality(Gnew)
#adding the meassure as node attribute to the graph object
nx.set_node_attributes(Gnew, centrality, "ClosenessCentrality")

In [ ]:
#plotting the closeness centrality of the network as histogram
plt.hist(list(close_centrality.values()))
plt.title("Histogram on Closeness Centrality Distribution")
plt.show()

In [ ]:
#plotting the graph with the node sizes depending on the closeness centrality messure of the nodes.
fig = plt.figure(1, figsize=(30,30))
nx.draw(Gnew, pos = pos, nodelist=close_centrality.keys(), node_color="seagreen", node_size=[v * 10000 for v in close_centrality.values()])
plt.show()

## Betweenness Centrality: 
*Betweenness centrality* is computed under the assumption that important nodes connect other nodes. Nodes with a high betweeness cetnrality act like bridges in the network. 

In [ ]:
#calculating the betweeness centrality:
bet_centrality = nx.betweenness_centrality(Gnew)
#adding the betweeness centrality as node attribute to the graph object: 
nx.set_node_attributes(Gnew, bet_centrality, "BetweenessCentrality")

In [ ]:
#plotting the degree distribution of the network
plt.hist(list(bet_centrality.values()))
plt.title("Histogramm on the Betweenness Centrality Distribution")
plt.show()

### Finding the Nodes with highest betweenness centrality
Defining the function *summary* that profides us with the nodes that have the highest betweeness centrality. 

In [ ]:
# defining the function summary
def summary(G):
    '''
    Getting the values and nodes with the highest betweeness centrality in descending order
    '''
    #use from_dic() to create a dataframe with the keys and vlaues of the bet_centrality object
    df = pd.DataFrame.from_dict({
        'node': list(bet_centrality.keys()),
        'between_centrality': list(bet_centrality.values())
    })
    #sort the values by centrality with descending order:
    return df.sort_values('between_centrality', ascending=False)

In [ ]:
top_bet_cent = summary(Gnew)
top_bet_cent.head()

Subsetting the graphobject on the top 100 nodes with the highest in betweeness centrality meassure. 

In [ ]:
#subsetting the dataframe, keeping only the top 100 nodes with the highest betweenness centrality. 
topbet = top_bet_cent.iloc[0:100]
topbet.head()

### Plotting the new graph object T. 

In [ ]:
#Creating new graph object as a subgraph og Gnew only with the top 100 nodes: 
topbet_nodes = topbet["node"]
T = Gnew.subgraph(topbet_nodes)

In [ ]:
#Plotting the new graph object:
npos = nx.spring_layout(T, scale=1)
fig = plt.figure(1, figsize=(30,30))
nx.draw(T, pos=npos, node_color="seagreen", with_labels = True, node_size=500, edge_color="black")
plt.show()

Plotting the graphobject T with nodes size depending on the betweenness centrality meassure. 

In [ ]:
#calculating the betweenness centrality of the graph object T
between_centrality_T = nx.betweenness_centrality(T)
#plotting T with node_size depending on betweenness centrality
npos = nx.spring_layout(T, scale=1)
fig = plt.figure(1, figsize=(20,20))
nx.draw(T, pos=npos, node_color="seagreen", 
        node_size=[v * 10000 for v in between_centrality_T.values()], 
        #also adding labels
        with_labels=True,
        edge_color="black")
plt.show()

### Defining a function *find_node_with_highest_bet_cent* 
The function returns the nodes with the highest betweeness centrality in the graph G.

In [ ]:
# Define find_node_with_highest_bet_cent()
def find_node_with_highest_bet_cent(G):
    '''returns the nodes with the highest betweeness centrality in the graph G.'''
    # Computing betweenness centrality: bet_cent
    bet_cent = nx.betweenness_centrality(G)
    # Computing maximum betweenness centrality: max_bc
    max_bc = max(list(bet_cent.values()))
    nodes = set()
    # Iterating over the betweenness centrality dictionary
    for k, v in bet_cent.items():
        # Checking if the current value has the maximum betweenness centrality
        if v == max_bc:
            # Adding the current node to the set of nodes
            nodes.add(k)
    return nodes

# Using that function to find the node(s) that has the highest betweenness centrality in the network: top_bc
top_bc = find_node_with_highest_bet_cent(Gnew)
print("This is the node with the highest betweeness centrality: ",top_bc)

# Assertion statement that checks that the node(s) is/are correctly identified.
for node in top_bc:
    assert nx.betweenness_centrality(Gnew)[node] == max(nx.betweenness_centrality(Gnew).values())

## Neihgbors of nodes
Getting the degree of every nodes in the graph. Plotting a histogram of the degree centrality distribution. 

In [ ]:
#computing the degree of every node: degrees
degrees = [len(list(Gnew.neighbors(n))) for n in Gnew.nodes()]
print(degrees)

In [ ]:
#Plot a histogram of the degree distribution of the graph
plt.figure()
plt.hist(degrees)
plt.title("Histogram of the degree distribution of the graph")
plt.show()

In [ ]:
#Plot a scatter plot of the centrality distribution and the degree distribution
plt.figure()
plt.scatter(degrees, list(centrality.values()))
plt.title("Scatter Plot of the centrality and the degree distribution")
plt.show()

#### Defing the function *nodes_neighbors*
The function *nodes_neighbors*  returns the count of neighbors the nodes have. 

In [ ]:
#defining the function nodes_neighbors
def nodes_neighbors(G):
    """
    Returns the count of neighbors the nodes in G have.
    """
    nodes = set()
    #iterate over all nodes in G
    for n in G.nodes():
        #counting the nodes and adding them to the set:
        nodes.add(len(list(G.neighbors(n))))
        
    #return the neighbor of n
    return nodes


In [ ]:
neighbors = nodes_neighbors(Gnew)
print(neighbors)

#### Defining a functure *nodes_with_m_nrbs* 
The function return all nodes in a graph that have a soecific count of (m) neighbors.

In [ ]:
# Define nodes_with_m_nbrs()
def nodes_with_m_nbrs(G, m):
    """
    Returns all nodes in graph G that have m neighbors.
    """
    nodes = set()

    # Iterate over all nodes in G
    for n in G.nodes():

        # Check if the number of neighbors of n matches m
        if len(list(G.neighbors(n))) == m:

            # Add the node n to the set
            nodes.add(n)

    # Return the nodes with m neighbors
    return nodes

In [ ]:
# Compute and print all nodes in Gnew that have 6 neighbors
six_nbrs = nodes_with_m_nbrs(Gnew, 6)
print(six_nbrs)


In [ ]:
# Compute and print all nodes in Gnew that have 72 neighbors
seventytwo_nbrs = nodes_with_m_nbrs(Gnew, 72)
print(seventytwo_nbrs)

In [ ]:
#checking if there realy is no node with 14 neighbors 
#(As the result shows when using the nodes_neighbors function on Gnew): 
fourteen_nbrs = nodes_with_m_nbrs(Gnew, 14)
print(fourteen_nbrs)

## Investigate Triangles in the graph
Finding nodes that are involved in triangles. The API nx.triangles() returns a dictionary where the nodes are the keys and the values are the number of triangles. 

In [ ]:
#creating a dictionary with the triangles of nodes. 
triangles = nx.triangles(Gnew)

In [ ]:
triangles.get(1)
#nothing is return, there is no node with 1 triangle. 

In [ ]:
triangles_values = list(triangles.values())
print(sorted(triangles_values))

In [ ]:
#complicate way to find out if there are tringles. 
triangles_values = list(triangles.values())
triangles2 = [i for i in triangles_values if i > 1]
print(triangles2)

There are no triangles in the graph object Gnew.Therefore tThe following function is not useful in this case, since there are no nodes in a triangle relationships.
Code was: 
from itertools import combinations
#Function that identifies all nodes in a triangle relationship.
def nodes_in_triangle(G, n):
    """
    Returns the nodes in a graph G that are involved in a triangle relationship.
    """
    triangle_nodes = set([n])
    # Iterating over all possible triangle relationship combinations
    for n1, n2 in combinations(G.neighbors(n), 2):
        # Checking if n1 and n2 have an edge between them
        if G.has_edge(n1, n2) == True:
            # Adding n1 to triangle_nodes
            triangle_nodes.add(n1)
            # Adding n2 to triangle_nodes
            triangle_nodes.add(n2)
    return triangle_nodes

nodes_in_triangle(Gnew, 1)
#shouldn't the result show at least two nodes? 

## Cliques

In [ ]:
#finding all cliques. not very informative in this case. Only holds pairs of two nodes. 
cliques = list(nx.find_cliques(Gnew))
print(cliques[:5])

##### Finding cliques.  
cliques are "groups of nodes that are fully connected to one another", while a maximal clique is a clique that cannot be extended by adding another node in the graph.

In [ ]:
#finding the cliques: 
cliques = nx.find_cliques(Gnew)
cliques

In [ ]:
print("There are", len(list(cliques)), "cliques in the graph")
#why do I get the result 0?

In [ ]:
largest_clique = sorted(nx.find_cliques(Gnew), key=lambda x:len(x))[-1]
print("The largest clique constists of the nodes: ", largest_clique)

When the largest clique constits of only two nodes, there are no real cliques. 